# Onchain Attest Labels Using Python

### Single Attest

In [ ]:
from web3 import Web3
from eth_abi.abi import encode
import json

# Connect to an Ethereum node
rpc = 'https://sepolia.base.org' # for Base use: https://mainnet.base.org
web3 = Web3(Web3.HTTPProvider(rpc))

# Check if connected to the node
if not web3.is_connected():
    raise Exception("Failed to connect to the Ethereum node")

# Load the EAS contract
from eas_contracts import eas_address, eas_abi
eas = web3.eth.contract(address=eas_address, abi=eas_abi)

# Import your wallet
address = '...' #### fill in here!
private_key = '...' #### fill in here!

# Label Pool Schema v1.0.0
schema = '0xb763e62d940bed6f527dd82418e146a904e62a297b8fa765c9b3e1f0bc6fdd68'

# Input the label in OLI format:
#   chain_id should follow the CAIP-2 standard
#   contract_address should be the address of the contract you want to label
#   tags_json should be a JSON string
chain_id = 'eip:8453'
contract_address = '0x498581ff718922c3f8e6a244956af099b2652b2b'
ref_uid = '0x0000000000000000000000000000000000000000000000000000000000000000'
tags_json = json.dumps(
                {'contract_name': 'Pool Manager v4',
                'project': 'PoolTogether',
                'is_eoa': False, 
                'deployment_tx': '0x25f482fbd94cdea11b018732e455b8e9a940b933cabde3c0c5dd63ea65e85349',
                'deployer_address': '0x2179a60856E37dfeAacA0ab043B931fE224b27B6',
                'owner_project': 'uniswap',
                'version': 4,
                'deployment_date': '2025-01-21 20:28:43',
                'source_code_verified': 'https://repo.sourcify.dev/contracts/partial_match/8453/0x498581fF718922c3f8e6A244956aF099B2652b2b/',
                'is_proxy': False}
            )

# ABI encode
encoded_data = encode(['string', 'string'], [chain_id, tags_json])
data = f"0x{encoded_data.hex()}"

# Create the transaction to call the attest function
transaction = eas.functions.attest({
    'schema': web3.to_bytes(hexstr=schema),
    'data': {
        'recipient': web3.to_checksum_address(contract_address),
        'expirationTime': 0,
        'revocable': True,
        'refUID': web3.to_bytes(hexstr=ref_uid),
        'data': web3.to_bytes(hexstr=data),
        'value': 0
    }
}).build_transaction({
    'chainId': 84532, # for Base use: 8453 
    'gas': 1000000,
    'gasPrice': web3.eth.gas_price,
    'nonce': web3.eth.get_transaction_count(address),
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the transaction
txn_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Get the transaction receipt
txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)

print(f'Transaction successful with hash: 0x{txn_hash.hex()}')

Transaction successful with hash: 0xe6cf4e7ceb2589940e7d9f86d60be3124c942bb8d2d82b01939fe86fa9d188cd


### Calculate UID

In [3]:
def calculate_attestation_uid(schema, recipient, attester, timestamp, data, expiration_time=0, revocable=True, ref_uid=ref_uid, bump=0):
    # Convert all values to bytes
    schema_bytes = Web3.to_bytes(hexstr=schema)
    recipient_bytes = Web3.to_bytes(hexstr=recipient)
    attester_bytes = Web3.to_bytes(hexstr=attester)
    ref_uid_bytes = Web3.to_bytes(hexstr=ref_uid)
    data_bytes = Web3.to_bytes(hexstr=data)
    revocable_bytes = bytes([1]) if revocable else bytes([0])
    timestamp_bytes = timestamp.to_bytes(8, byteorder='big')
    expiration_bytes = expiration_time.to_bytes(8, byteorder='big')
    bump_bytes = bump.to_bytes(4, byteorder='big') 
    # Pack all the values together in the same order as the Solidity contract
    packed_data = (schema_bytes + recipient_bytes + attester_bytes + timestamp_bytes + expiration_bytes + revocable_bytes + ref_uid_bytes + data_bytes + bump_bytes)
    # Calculate keccak256 hash
    uid = Web3.keccak(packed_data)
    return uid

# get the timestamp from the transaction receipt
timestamp = web3.eth.get_block(txn_receipt['blockNumber'])['timestamp']

# Calculate UID (legacy)
uid = calculate_attestation_uid(schema, contract_address, address, timestamp, data)
uid_hex = '0x' + uid.hex()

print(f"Calculated UID: {uid_hex}")

Calculated UID: 0x0964fa16b14cd86b3e83187ba43fa6921d30ddf228084e15e1c029c556600ceb


In [4]:
# getAttestation
eas.functions.getAttestation(web3.to_bytes(hexstr=uid_hex)).call()

(b'\td\xfa\x16\xb1L\xd8k>\x83\x18{\xa4?\xa6\x92\x1d0\xdd\xf2(\x08N\x15\xe1\xc0)\xc5V`\x0c\xeb',
 b'\xb7c\xe6-\x94\x0b\xedoR}\xd8$\x18\xe1F\xa9\x04\xe6*){\x8f\xa7e\xc9\xb3\xe1\xf0\xbco\xddh',
 1741775086,
 0,
 0,
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00',
 '0x498581fF718922c3f8e6A244956aF099B2652b2b',
 '0x732D31D49467c08F41fD0727537995ea45dD4Ba7',
 True,
 b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00@\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x80\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x08eip:8453\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\

In [5]:
# isAttestationValid
eas.functions.isAttestationValid(web3.to_bytes(hexstr=uid_hex)).call()

True

### Single Revoke

In [6]:
# Create the transaction to revoke an attestation
transaction = eas.functions.revoke({
    'schema': web3.to_bytes(hexstr=schema),
    'data': {
        'uid': web3.to_bytes(hexstr=uid_hex),
        'value': 0
    }
}).build_transaction({
    'chainId': 84532, # for Base use: 8453 
    'gas': 1000000,
    'gasPrice': web3.eth.gas_price,
    'nonce': web3.eth.get_transaction_count(address),
})

# Sign the transaction
signed_txn = web3.eth.account.sign_transaction(transaction, private_key=private_key)

# Send the transaction
txn_hash = web3.eth.send_raw_transaction(signed_txn.raw_transaction)

# Get the transaction receipt
txn_receipt = web3.eth.wait_for_transaction_receipt(txn_hash)

print(f'Transaction successful with hash: 0x{txn_hash.hex()}')

Transaction successful with hash: 0x71e81b77c9b22851ec8a8e8a074af4a24a3916bef3a218003784124b9c6cbe50
